In [2]:
import pandas as pd
import numpy as np
import os
from agents import Runner, agent, trace, set_trace_processors, Agent, ModelSettings
from weave.integrations.openai_agents.openai_agents import WeaveTracingProcessor
import asyncio
from typing import Literal, List, Annotated
import weave
from dotenv import load_dotenv
from pydantic import BaseModel, Field, conlist
from IPython.display import display, Markdown, HTML, Image
from tqdm import tqdm
import warnings
load_dotenv()
# Import Necessary Libraries
from src.Agents.Master_Agent import q_a_agent
# Import Weave and Biases
weave.init("Operation Model")
set_trace_processors([WeaveTracingProcessor()])
warnings.filterwarnings("ignore")

weave: Logged in as Weights & Biases user: francisco-reveriano-1.
weave: View Weave data at https://wandb.ai/francisco-reveriano-1-mckinsey-company/operation-model/weave


## Load Dataset

In [20]:
excel_sheet = pd.ExcelFile("../Data/Raw/Ops_Overview_Data_File.xlsx")
# List all sheet names
print(excel_sheet.sheet_names)

['Original HC Move Table', 'Mortgage Servicing', 'Sheet7', 'FTS', 'Client Default Solutions', 'Truist Care Centers', 'Enterprise Resilience', 'Deposits Service Delivery', 'Wholesale Banking - Cap Mar', 'Wholesale Banking - Lending', 'Scenarios_vArchive', 'CSBB Service Delivery', 'Supplier Spend Data']


In [21]:
df = None
df = pd.read_excel("../Data/Raw/Ops_Overview_Data_File.xlsx", sheet_name="Wholesale Banking - Cap Mar")
df.fillna(0, inplace=True)
df["ONSHORE"] = df[['ONSHORE TEAMMATE', 'ONSHORE CW', 'Est. Size- ONSHORE ']].sum(axis=1)
df["OFFSHORE"] = df[["Est. Size- OFFSHORE"]]
#df["ONSHORE"] = df["Est. Size- ONSHORE "]
#df["OFFSHORE"] = df["Est. Size- OFFSHORE"]
print(df.shape)
df.head(20)

(23, 9)


,Function / Process Owner (L6),Function / Process Owner (L7),Function / Process Name,ONSHORE TEAMMATE,ONSHORE CW,Est. Size- ONSHORE,Est. Size- OFFSHORE,ONSHORE,OFFSHORE
0,John Milligan,0,Directs,0.0,0.0,5.0,0.0,5.0,0.0
1,Ed Kelley,Jon Harsch,Derivative Confirmations,0.0,0.0,4.0,0.0,4.0,0.0
2,0,Rich Tyson,Derivative Settlements,0.0,0.0,5.0,0.0,5.0,0.0
3,0,Rob Pogorzelski,Collateral Management,0.0,0.0,3.0,0.0,3.0,0.0
4,0,Steve Gerstel,SD Regulatory Trade Reporting,0.0,0.0,6.0,0.0,6.0,0.0
5,0,Chris Wildes,Derivative Confirmations,0.0,0.0,1.0,0.0,1.0,0.0
6,0,0,RAPTOR - CAPCO,0.0,3.0,0.0,0.0,3.0,0.0
7,0,0,RAPTOR - Deloitte,0.0,2.0,0.0,0.0,2.0,0.0
8,Melissa Matney,John Cadogan,BD Funding / Corp Actions,0.0,0.0,3.0,12.0,3.0,12.0
9,0,Andrew Pashayan,BD Trade Support,0.0,0.0,8.0,3.0,8.0,3.0


In [4]:
sheet_name = "FTS"
for index, row in df.iterrows():
    process   = row["Function / Process Name"]
    onshore   = row["ONSHORE"]
    offshore  = row["OFFSHORE"]

     # Build the two prompts
    onshore_query = (
        f"What is the 5-year cumulative percentage change that Generative AI will have "
        f"on the following process: {process} within the following Line of Business (LoB): "
        f"{sheet_name}? Keep in mind that the bank's current onshore capacity is {onshore} Full-Time Equivalent Employees."
    )
    offshore_query = (
        f"What is the 5-year cumulative percentage change that Generative AI will have "
        f"on the following process: {process} within the following Line of Business (LoB): "
        f"{sheet_name}? Keep in mind that the bank's current offshore capacity is {offshore} Full-Time Equivalent Employees."
    )
    on = await Runner.run(q_a_agent, onshore_query, max_turns=100)
    df.loc[index, "Onshore_High_Scenario_Vectors"] =    on.final_output.high_scenario
    df.loc[index, "Onshore_Medium_Scenario_Vectors"] =  on.final_output.medium_scenario
    df.loc[index, "Onshore_Low_Scenario_Vectors"] =     on.final_output.low_scenario
    df.loc[index, "Onshore_High_Scenario_Reasoning"] =  on.final_output.high_scenario_reasoning
    df.loc[index, "Onshore_Medium_Scenario_Reasoning"] = on.final_output.medium_scenario_reasoning
    df.loc[index, "Onshore_Low_Scenario_Reasoning"] =   on.final_output.low_scenario_reasoning
    df.loc[index, "Onshore_CourseWork"] =               on.final_output.online_coursework
    of = await Runner.run(q_a_agent, offshore_query, max_turns=100)
    df.loc[index, "Offshore_High_Scenario_Vectors"] =    of.final_output.high_scenario
    df.loc[index, "Offshore_Medium_Scenario_Vectors"] =  of.final_output.medium_scenario
    df.loc[index, "Offshore_Low_Scenario_Vectors"] =     of.final_output.low_scenario
    df.loc[index, "Offshore_High_Scenario_Reasoning"] =  of.final_output.high_scenario_reasoning
    df.loc[index, "Offshore_Medium_Scenario_Reasoning"] = of.final_output.medium_scenario_reasoning
    df.loc[index, "Offshore_Low_Scenario_Reasoning"] =   of.final_output.low_scenario_reasoning
    df.loc[index, "Offshore_CourseWork"] =               of.final_output.online_coursework
    df.to_excel("../Data/Intermediate/Ops_Overview_Data_File_Processed.xlsx", sheet_name=sheet_name, index=False)

df.to_excel("../Data/Intermediate/Ops_Overview_Data_File_{}_Processed.xlsx".format(sheet_name), sheet_name=sheet_name, index=False)


Function / Process Name    Lockbox / Business Support
ONSHORE TEAMMATE                                    5
ONSHORE CW                                          0
Est. Size- ONSHORE                                4.0
Est. Size- OFFSHORE                               0.0
ONSHORE                                           9.0
OFFSHORE                                          0.0
Name: 0, dtype: object


In [ ]:
df